<a href="https://colab.research.google.com/github/KwonHo-geun/automobile/blob/main/06.25.%EC%9E%90%EC%9C%A8%EC%A3%BC%ED%96%89_%EC%9E%90%EB%8F%99%EC%B0%A8_for%EB%AC%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
fruits = ["apple","banana","cherry"]
for x in fruits:
  print(x)

apple
banana
cherry


In [2]:
for x in "apple":
  print(x)

a
p
p
l
e


In [3]:
fruits = ["apple", "banana", "cherry"]
for x in fruits:
  print(x)
  if x == "banana":
    break


apple
banana


In [4]:
fruits = ["apple", "banana", "cherry"]
for x in fruits:
  if x == "banana":
    break
  print(x)

apple


In [5]:
fruits = ["apple", "banana", "cherry"]
for x in fruits:
  if x == "banana":
    continue
  print(x)

apple
cherry


In [7]:
distances = [110, 50, 60, 40, 90]

for distance in distances:
    if distance > 70:
        speed = 50
    elif distance > 40:
        speed = 30
    else:
        speed = 10
    print(f"거리: {distance}cm → 속도 조절: {speed} km")

거리: 110cm → 속도 조절: 50 km
거리: 50cm → 속도 조절: 30 km
거리: 60cm → 속도 조절: 30 km
거리: 40cm → 속도 조절: 10 km
거리: 90cm → 속도 조절: 50 km


In [8]:
lane_positions = ["중앙", "중앙", "왼쪽", "중앙", "오른쪽"]

for position in lane_positions:
    if position == "중앙":
        action = "유지"
    elif position == "왼쪽":
        action = "우회전 보정"
    else:
        action = "좌회전 보정"
    print(f"차량 위치: {position} → 조향 명령: {action}")


차량 위치: 중앙 → 조향 명령: 유지
차량 위치: 중앙 → 조향 명령: 유지
차량 위치: 왼쪽 → 조향 명령: 우회전 보정
차량 위치: 중앙 → 조향 명령: 유지
차량 위치: 오른쪽 → 조향 명령: 좌회전 보정


In [10]:
obstacles = [False, True, False, True, True] # 장애물 유무

for i, obstacle in enumerate(obstacles):
    if obstacle:
        print(f"{i+1}번째 체크포인트: 장애물 감지 → 오른쪽으로 회피")
    else:
        print(f"{i+1}번째 체크포인트: 주행 유지")

1번째 체크포인트: 주행 유지
2번째 체크포인트: 장애물 감지 → 오른쪽으로 회피
3번째 체크포인트: 주행 유지
4번째 체크포인트: 장애물 감지 → 오른쪽으로 회피
5번째 체크포인트: 장애물 감지 → 오른쪽으로 회피


In [11]:
target_speed = 60
current_speed = 0

for i in range(5):
    if current_speed < target_speed:
        current_speed += 20
    else:
        current_speed -= 10
    print(f"현재 속도: {current_speed} km/h")

현재 속도: 20 km/h
현재 속도: 40 km/h
현재 속도: 60 km/h
현재 속도: 50 km/h
현재 속도: 70 km/h


In [9]:
route = ["신호등", "횡단보도", "빈 도로", "장애물"]

for event in route:
    if event == "신호등":
        print("신호등 감지 → 신호 확인")
    elif event == "횡단보도":
        print("횡단보도 감지 → 보행자 확인")
    elif event == "장애물":
        print("장애물 감지 → 정지")
    else:
        print("정상 주행")

신호등 감지 → 신호 확인
횡단보도 감지 → 보행자 확인
정상 주행
장애물 감지 → 정지


In [ ]:
import rospy
import cv2
import numpy as np
from std_msgs.msg import Float32
from carla_msgs.msg import CarlaEgoVehicleControl

class SpeedBumpDetector:
    def __init__(self):
        # 비전 센서 초기화 (실제 하드웨어 연동 시 카메라 설정 추가)
        self.cap = cv2.VideoCapture(0)  # 기본 카메라 장치
        self.bump_detected = False
        self.bump_distance = 0.0

    def detect_speed_bump(self):
        """과속 방지턱 감지 및 거리 계산"""
        ret, frame = self.cap.read()
        if not ret:
            return False, 0.0

        # 영상 처리: 색상 기반 검출 (실제 구현에선 ML/DNN 모델 사용)
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        lower_yellow = np.array([20, 100, 100])
        upper_yellow = np.array([30, 255, 255])
        mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

        # 감지 결과 처리
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            max_contour = max(contours, key=cv2.contourArea)
            x, y, w, h = cv2.boundingRect(max_contour)

            # 거리 추정 (픽셀 좌표 기반)
            self.bump_distance = self.estimate_distance(w)
            self.bump_detected = True
            return True, self.bump_distance

        self.bump_detected = False
        return False, 0.0

    def estimate_distance(self, width):
        """물체 너비 기반 거리 계산"""
        # 실제 구현에선 카메라 캘리브레이션 파라미터 사용
        focal_length = 500  # 카메라 초점거리 (실제 값으로 교체)
        known_width = 0.3   # 과속 방지턱 표준 너비(m)
        return (known_width * focal_length) / width

class SpeedController:
    def __init__(self, kp=0.5, ki=0.01, kd=0.1):
        # PID 제어기 파라미터
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.prev_error = 0.0
        self.integral = 0.0
        self.safe_speed = 30.0  # km/h (과속 방지턱 근처 권장 속도)

    def calculate_throttle(self, current_speed, bump_detected, bump_distance):
        """과속 여부 판단 및 스로틀 계산"""
        if not bump_detected:
            return 0.7  # 기본 주행 스로틀

        # 과속 판단 로직
        safe_distance = 15.0  # 감속 시작 거리(m)
        if bump_distance < safe_distance:
            # PID 제어를 통한 감속
            error = self.safe_speed - current_speed
            self.integral += error
            derivative = error - self.prev_error
            throttle = self.kp * error + self.ki * self.integral + self.kd * derivative
            self.prev_error = error
            return max(0.0, min(throttle, 1.0))  # 0~1 범위 제한

        return 0.7  # 안전 거리 밖이면 기본 속도 유지

def main():
    rospy.init_node('autonomous_speed_control')

    # ROS 설정
    speed_pub = rospy.Publisher('/vehicle/speed_control', CarlaEgoVehicleControl, queue_size=10)
    speed_sub = rospy.Subscriber('/carla/ego_vehicle/speedometer', Float32, speed_callback)

    # 모듈 초기화
    detector = SpeedBumpDetector()
    controller = SpeedController()

    current_speed = 0.0

    def speed_callback(msg):
        nonlocal current_speed
        current_speed = msg.data * 3.6  # m/s → km/h 변환

    rate = rospy.Rate(10)  # 10Hz 제어 주기

    while not rospy.is_shutdown():
        # 1. 과속 방지턱 감지
        bump_detected, bump_distance = detector.detect_speed_bump()

        # 2. 과속 판단 및 제어
        control_msg = CarlaEgoVehicleControl()
        control_msg.throttle = controller.calculate_throttle(
            current_speed, bump_detected, bump_distance
        )

        # 3. 제어 명령 발행
        speed_pub.publish(control_msg)
        rate.sleep()

if __name__ == "__main__":
    main()


In [ ]:
# 기본 비디오 처리 루프
while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


In [ ]:
# 각 탐지 결과 순회
for result in results:
    boxes = result.boxes
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = box.xyxy[0]
        conf = box.conf[0]
        cls = box.cls[0]



In [ ]:
for i, (x1, y1, x2, y2) in enumerate(boxes):
    cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

In [ ]:
for i in range(len(boxes)):
    if conf[i] > 0.5:
        # 처리 로직
        pass
